# Exercise 8: Sound transformations

In this exercise you will use the HPS model to creatively transform sounds. There are two parts in this exercise. In the first one you should perform a natural sounding transformation on the speech sound that you used in the previous exercise (E7). In the second part you should select a sound of your choice and do a "creative" transformation. You will have to write a short description of the sound and of the transformation you did, giving the link to the original sound and uploading several transformed sounds.

For this exercise, you can use the `transformations_GUI.py` (in `software/transformations_interface/`) to try things, once decided you can fill up the code in this file. You can also do everything from here and add any new code you wish.

In order to perform a good/interesting transformation you should make sure that you have performed an analysis that is adequate for the type of transformation you want to do. Not every HPS analysis representation will work for every type of sound transformation. There will be things in the analysis that when modified will result in undesired artifacts. In general, for any transformation, it is best to have the harmonic values as smooth and continuous as possible and an stochastic representation as smooth and with as few values as possible. It might be much better to start with an analysis representation that does not result in the best reconstruction in exchange of having smoother and more compact data.

To help you with the exercise, we give a brief description of the transformation parameters used by the HPS transformation function:

1. `freqScaling`: frequency scaling factors to be applied to the harmonics of the sound, in time-value pairs (where value of 1 is no scaling). The time values can be normalized, from 0 to 1, or can correspond to the times in seconds of the input sound. The scaling factor is a multiplicative factor, thus a value of 1 is no change. Example: to transpose an octave the sound you can specify `[0, 2, 1, 2]`.
2. `freqStretching`: frequency stretching factors to be applied to the harmonics of the sound, in time-value pairs (value of 1 is no stretching). The time values can be normalized, from 0 to 1, or can correspond to the times in seconds of the input sound. The stretching factor is a multiplicative factor whose effect depend on the harmonic number, higher harmonics being more affected that lower ones, thus resulting in an inharmonic effect. A value of 1 results in no transformation. Example: an array like `[0, 1.2, 1, 1.2]` will result in a perceptually large inharmonic effect.
3. `timbrePreservation`: 1 preserves the original timbre, 0 does not. It can only have a value of 0 or of 1. By setting the value to 1 the spectral shape of the original sound is preserved even when the frequencies of the sound are modified. In the case of speech it would correspond to the idea of preserving the identity of the speaker after the transformation.
4. `timeScaling`: time scaling factors to be applied to the whole sound, in time-value pairs (value of 1 is no scaling). The time values can be normalized, from 0 to 1, or can correspond to the times in seconds of the input sound. The time scaling factor is a multiplicative factor, thus 1 is no change. Example: to stretch the original sound to twice the original duration, we can specify `[0, 0, 1, 2]`.

All the transformation values can have as many points as desired, but they have to be in the form of an array with time-value pairs, so of even size. For example a good array for a frequency stretching of a sound that has a duration of 3.146 seconds could be: `[0, 1.2, 2.01, 1.2, 2.679, 0.7, 3.146, 0.7]`.

## Part 1. Perform natural sounding transformations of a speech sound

Use the HPS model with the sound `speech-female.wav`, available in the sounds directory, to first analyze and then obtain a natural sounding transformation of the sound. The synthesized sound should sound as different as possible to the original sound while sounding natural. By natural we mean that it should sound like speech, that it could have been possible to be produced by a human, and by listening we should consider it as a speech sound, even though we might not be able to understand it. You should first make sure that you start from a good analysis, then you can do time and/or frequency scaling transformations. The transformation should be done with a single pass, no mixing of sounds coming from different transformations. Since you used the same sound in A7, use that experience to get a good analysis, but consider that the analysis, given that we now want to use it for applying a very strong transformation, might be done differently than what you did in A7.

Write a short paragraph for every transformation, explaining what you wanted to obtain and explaining the transformations you did, giving both the analysis and transformation parameter values (sufficiently detailed for the evaluator to be able to reproduce the analysis and transformation).

In [1]:
import numpy as np
from scipy.signal import get_window
import matplotlib.pyplot as plt
import IPython.display as ipd

from smstools.models import utilFunctions as UF
from smstools.models import stft as STFT
from smstools.models import hpsModel as HPS
from smstools.transformations import hpsTransformations as HPST
from smstools.transformations import harmonicTransformations as HT
import IPython.display as ipd

In [73]:
# 1.1 perform an analysis/synthesis using the HPS model

input_file = '../sounds/speech-female.wav'

### fill the parameters
window = 'blackmanharris'
M = 2048
N = 4096
t = -120        
minSineDur = 0.1
nH = 50        
minf0 = 100   
maxf0 = 300   
f0et = 10       
harmDevSlope = 0.01
stocf = 0.2   

# no need to modify anything after this
Ns = 512
H = 128

(fs, x) = UF.wavread(input_file)
w = get_window(window, M, fftbins=True)
hfreq, hmag, hphase, stocEnv = HPS.hpsModelAnal(x, fs, w, N, H, t, nH, minf0, maxf0, f0et, harmDevSlope, minSineDur, Ns, stocf)
y, yh, yst = HPS.hpsModelSynth(hfreq, hmag, hphase, stocEnv, Ns, H, fs)

ipd.display(ipd.Audio(data=x, rate=fs))
ipd.display(ipd.Audio(data=y, rate=fs))

**Question E8 - 1.2:**

explain your parameter choices in 1.2

The final parameter set for analyzing speech-female.wav was chosen to achieve an optimal trade-off between maximum data compression and the preservation of perceptual quality of the human voice. This approach is paramount in speech coding, where intelligibility and natural timbre must be maintained, even with a reduced data representation. This strategy is reflected in key parameter choices, such as the moderate harmonic limit (nH=50) and the compression of the stochastic residual (stocf=0.2), actively pursuing compacting without introducing noticeable auditory artifacts.

The Blackman-Harris window is selected for its high side-lobe suppression, which minimizes spectral leakage to yield cleaner harmonic peaks. A large window size (M=2048) is used to provide high frequency resolution, ensuring accurate separation and estimation of harmonics, with the FFT size (N=4096) set to twice M for optimal peak estimation.

The f0​ range is constrained between 100 Hz and 300 Hz, covering the female vocal register, and the flexible error threshold (f0et=10) is chosen to reliably maintain f0​ tracking across the rapid pitch variations characteristic of speech. The very low peak threshold (t=−120 dB) is employed to ensure the capture of all subtle harmonic components, preserving the richness of the voice's timbre.

To balance data compression with perceptual quality, the maximum number of harmonics (nH) is limited to 50. This moderate number reduces the sinusoidal data size while still capturing the necessary partials to define the voice's crucial vowel formants. A long minimum sine duration (minSineDur=0.1 s) filters out unstable, transient harmonic tracks, improving harmonic compacting. Finally, the stochastic decimation factor (stocf=0.2) is chosen to create a compact and smoothed spectral envelope of the residual noise, effectively compressing the representation of unvoiced segments (fricatives and sibilants) while maintaining perceptual detail.
________

In [74]:
### define the transformations

# Pitch: Empieza bajo (0.88) y cae un poco más al final (0.82) por "cansancio".
# No bajo de 0.8 para que no suene masculino.
freqScaling = np.array([0, 0.88, 1, 0.82]) 

# Inharmonicity: 1.0 (La voz debe seguir siendo armónica y natural).
freqStretching = np.array([0, 1.0, 1, 1.0]) 

# Timbre: 1 (Obligatorio para mantener la identidad de "mujer" pese al tono grave).
timbrePreservation = 1

# Time: 1.3 (Un 30% más lento, simula la lentitud motora al hablar).
timeScaling = np.array([0, 0, 1, 1.3])


# no need to modify the following code 
Ns = 512
H = 128

# frequency scaling of the harmonics 
hfreqt, hmagt = HT.harmonicFreqScaling(hfreq, hmag, freqScaling, freqStretching, timbrePreservation, fs)

# time scaling the sound (Using your optimized/corrected hpsTimeScale function)
yhfreq, yhmag, ystocEnv = HPST.hpsTimeScale(hfreqt, hmagt, stocEnv, timeScaling)

# synthesis from the trasformed hps representation 
y, yh, yst = HPS.hpsModelSynth(yhfreq, yhmag, np.array([]), ystocEnv, Ns, H, fs)

ipd.display(ipd.Audio(data=y, rate=fs))

**Question E8 1-4:**

explain your transformations

In this part of the exercise, the objective was to transform the original female speech recording into a distinct yet natural-sounding voice, specifically aiming for the characteristics of an elderly woman. To achieve this, the Harmonic plus Stochastic analysis was first optimized with a short minimum sine duration to accurately capture the transient details of the consonants. The transformation strategy involved a combination of pitch shifting and time stretching to simulate the physiological effects of aging. The frequency scaling was set to gradually decrease from 0.88 to 0.82, lowering the fundamental frequency to mimic the loss of vocal fold tension and fatigue associated with older age, while the time scaling factor was set to 1.3 to replicate a slower, more deliberate speaking cadence. Crucially, the timbre preservation parameter was set to 1, which ensured that while the pitch dropped, the spectral envelope and formant structure of the original female vocal tract remained intact, preventing the voice from sounding masculine or artificial. The result is a synthesized voice that sounds significantly different from the original younger speaker but retains a completely natural, human quality.
___

## Part 2. Perform creative transformations with a sound of your choice

Pick any natural and harmonic sound from Freesound and use the HPS model to do the most creative and interesting transformation you can come up with. Sounding as different as possible from the original sound.

It is essential that you start with a natural harmonic sound. Examples include (but not limited to) any acoustic harmonic instrument, speech, harmonic sound from nature, etc. As long as they have a harmonic structure, you can use it. You can even reuse the sound you used in A7-Part2 or upload your own sound to freesound and then use it.

The sound from Freesound to use could be in any format, but to use the sms-tools software you will have to first convert it to be a monophonic file (one channel), sampling rate of 44100, and 16bits samples.

You can do any interesting transformation with a single pass. It is not allowed to mix sounds obtained from different transformations. The transformed sound need not sound natural. So, time to show some creativity!

Write a short paragraph for every transformation, explaining what you wanted to obtain and explaining the transformations you did, giving both the analysis and transformation parameter values (sufficiently detailed for the evaluator to be able to reproduce the analysis and transformation).

In [6]:
# 2.1 perform an analysis/synthesis using the HPS model

### set the parameters
input_file = '../sounds/guitarcleanmelody.wav' 
window = 'blackmanharris'
M = 2048    
N = 4096
t = -120                      # Umbral muy bajo para conservar las armónicas débiles, crucial para el brillo y el realismo de la guitarra.
minSineDur = 0.1              # Duración mínima larga para asegurar que solo se capturan las notas estables, eliminando armónicas de ataques y transitorios rápidos.
nH = 50                       # Limita el número máximo de armónicos a 50, asegurando una representación compacta, ya que la f0 alta (<500 Hz) reduce la cantidad de armónicos relevantes.
minf0 = 500                   # Límite inferior para f0 por encima de la nota más baja (C#5 ≈ 554 Hz), eliminando ruido de baja frecuencia.
maxf0 = 900                   # Límite superior para f0 que cubre la nota más alta (G#5 ≈ 831 Hz) con margen de seguridad.
f0et = 5                      # Umbral de error estricto en la detección de f0 para evitar saltos o errores tonales, manteniendo la precisión.
harmDevSlope = 0.01           # Pendiente positiva que permite una desviación armónica ligeramente mayor en las frecuencias altas, manteniendo el realismo.
stocf = 0.2                   # Factor de diezmado bajo para obtener una envolvente residual compacta y detallada, crucial para capturar el ataque de púa.

# no need to modify anything after this
Ns = 512
H = 128

(fs, x) = UF.wavread(input_file)
w = get_window(window, M, fftbins=True)
hfreq, hmag, hphase, stocEnv = HPS.hpsModelAnal(x, fs, w, N, H, t, nH, minf0, maxf0, f0et, harmDevSlope, minSineDur, Ns, stocf)
y, yh, yst = HPS.hpsModelSynth(hfreq, hmag, hphase, stocEnv, Ns, H, fs)

ipd.display(ipd.Audio(data=x, rate=fs))
ipd.display(ipd.Audio(data=y, rate=fs))

**Question E8 2.2:**

explain your parameter choices

The selection of these HPS parameters is optimized for analyzing a high-pitched, clean guitar melody with the dual goals of high reconstruction fidelity and data compacting. The Blackman-Harris window is chosen for its superior spectral definition, which helps isolate the harmonics clearly. A long window size (M=2048) and corresponding FFT size (N=4096) are necessary to provide the high frequency resolution needed to accurately capture the subtle, stable harmonic structure that defines the guitar's clean timbre. Due to the high f0​ range (500 Hz to 900 Hz), the minf0 and maxf0 bounds are set high to restrict the search space, drastically improving the accuracy of the f0​ detection algorithm, which is further refined by a strict f0et of 5. The very low threshold (t=−120 dB) is crucial for preserving the quiet, high-frequency harmonics that contribute to the brightness. For compacting the sinusoidal data, a long minimum sine duration (minSineDur=0.1 s) filters out any brief, noisy harmonic tracks, ensuring only stable pitches are stored, while the maximum number of harmonics (nH=50) is capped to reflect the limited number of partials available due to the high f0​. Finally, the stochastic decimation factor (stocf=0.2) is chosen to create a smooth, compact envelope that effectively models the non-pitched noise of the finger and pick attacks, which are perceptually vital for the rhythm and articulation.

Perceptually, the reconstruction is nearly identical to the original sound, and based on the analysis, the peak picking threshold (t=−120 dB) proves to be the most decisive parameter (apart from the f0​ limits). This is because setting t this low ensures that the maximum number of subtle, high-frequency partials of the clean guitar are retained in the harmonic component, directly preserving the brightness and richness of the timbre that would otherwise be lost to the residual component if the threshold were higher.
___

In [ ]:
from scipy.interpolate import interp1d

# This function has been implemented with Gemini's help
def hpsTimeScale(hfreq, hmag, stocEnv, timeScaling):
    # 1. Calcular tiempos reales de salida (acumulando duraciones)
    t_in = timeScaling[::2]
    # np.diff calcula la duración de cada tramo, prepend=0 alinea el inicio
    t_out = np.cumsum(np.diff(t_in, prepend=0) * timeScaling[1::2])
    
    # 2. Calcular cuántos frames tendrá el resultado final
    L = hfreq.shape[0]
    new_L = int(L * t_out[-1] / t_in[-1])
    
    # 3. Crear mapa de índices: ¿Qué frame original toca leer ahora?
    # Mapeamos tiempo salida -> tiempo entrada -> índice de frame
    idxs = interp1d(t_out, t_in)(np.linspace(0, t_out[-1], new_L)) * (L - 1) / t_in[-1]
    
    # 4. Interpolar las 3 matrices de golpe (frecuencia, magnitud, estocástica)
    # Usamos interp1d vectorizado sobre el eje 0 (tiempo)
    gen_data = lambda arr: interp1d(np.arange(L), arr, axis=0, fill_value="extrapolate")(idxs)
    
    return gen_data(hfreq), gen_data(hmag), gen_data(stocEnv)

In [ ]:
# 2.3 Perform a transformation from the previous analysis

### define the transformations
freqScaling = np.array([0,1.09,1.95,1.09,1.96,0.81,3.28,0.81,3.29,0.86,3.53,0.86,3.54,0.96,3.78,0.96,3.82,0.91,4.84, 0.91, 4.85,0.65,5.13,0.65,5.14,1.08,5.99,1.08,6.02,0,6.22,0,8.1734,0])
freqStretching = np.array([0,1,1,1])
timbrePreservation = 1
timeScaling = np.array([
    0,    0.2,   
    1.95, 0.2,   

    1.96, 0.2,   
    3.28, 0.2,   

    3.29, 0.8,
    3.53, 0.8,
    
    3.54, 1.0,
    3.78, 1.0,

    3.82, 0.5,
    4.84, 0.5,
    
    4.85, 1.0,
    8.1734, 1.0  # Hasta el final
])

# no need to modify anything after this
Ns = 512
H = 128

# frequency scaling of the harmonics 
hfreqt, hmagt = HT.harmonicFreqScaling(hfreq, hmag, freqScaling, freqStretching, timbrePreservation, fs)

# time scaling the sound
yhfreq, yhmag, ystocEnv = hpsTimeScale(hfreqt, hmagt, stocEnv, timeScaling)

# synthesis from the trasformed hps representation 
y, yh, yst = HPS.hpsModelSynth(yhfreq, yhmag, np.array([]), ystocEnv, Ns, H, fs)

ipd.display(ipd.Audio(data=y, rate=fs))

**Question E8 - 2.4:**

explain your transformations

In this exercise, the original acoustic guitar sample was transformed to reproduce the recognizable jazz motif known as "the lick" associated with Adam Neely. The transformation relied heavily on the precise manipulation of the frequency scaling and time scaling arrays to remap the original pitch and duration of the guitar notes into the new melodic and rhythmic pattern. To determine the necessary timing values, the note onsets were identified manually by visually inspecting the waveform within the transformations_GUI tool. Furthermore, a critical part of the process involved the re-implementation of the hpsTimeScale function, as the provided algorithm contained logic errors that prevented it from handling the complex, variable time factors required for these specific rhythmic adjustments. The function was rewritten using a vectorized linear interpolation method to correctly map the input frames to the output duration, which solved the interpolation issues and allowed for a smoother reconstruction. Despite this technical improvement, the results demonstrated that while the pitch and timing were accurate, the extreme time compression required for the faster notes still introduced some perceptual artifacts in the stochastic component, slightly diminishing the naturalness of the plectrum attacks due to the resolution limits of the analysis windows.
___
